In [ ]:
import pandas as pd
import cv2
import os
from tqdm import tqdm
import numpy as np
from PIL import Image

from segmentation_ade import get_color_seg, get_pred_seg, preprocess_image

In [ ]:
source_dir = 'data/Most_extreme_pictures'
out_dir = 'data/out'

In [ ]:
def do_in_file_path(source_dir, action):
    # Recursively get all file paths
    file_paths = []
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if not file.startswith("Icon"):
                file_paths.append(os.path.join(root, file))

    # file_paths = [file_paths[0]]
    for file_path in tqdm(file_paths):
        tqdm.write(f'Processing: {file_path}')
        action(file_path)

In [ ]:
def preprocess_and_save(file_path):
    # Exclude non-image file types
    valid_extensions = ('.jpg', '.jpeg', '.png', '.tiff', '.bmp', '.gif')
    if file_path.lower().endswith(valid_extensions):
        relative_path = os.path.relpath(file_path, source_dir)

        # Ensure the destination file has a single .jpg extension
        relative_path = relative_path.split('.')[0] + '.jpg'
        destination_path = os.path.join(out_dir, 'images_preprocessed', relative_path)
        
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        
        image = cv2.imread(file_path)
        preprocessed_image = preprocess_image(image)
        cv2.imwrite(destination_path, preprocessed_image)

# Preprocess images and maintain subfolder structure
do_in_file_path(source_dir, preprocess_and_save)


In [ ]:
# Get segmentations
source_dir = f'{out_dir}/images_preprocessed/'

images_as_arrays = []
segmentation_destination_dir = f'{out_dir}/segmentations/'
os.makedirs(segmentation_destination_dir, exist_ok=True)


def get_and_save_segmentation(file_path):
        relative_path = os.path.relpath(file_path, source_dir)

        # Ensure the destination file has a single .jpg extension
        relative_path = relative_path.split('.')[0] + '.jpg'
        destination_path = os.path.join(segmentation_destination_dir, relative_path)
        
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        
        # Display the original image
        pil_image = Image.open(file_path)
        display(pil_image)

        image = np.array(pil_image)

        images_as_arrays.append({
                'img_name': os.path.basename(file_path),
                'predicted_segmentation': get_pred_seg(np.array(image)).numpy()
        })  

        # Get the color segmentation
        color_seg = get_color_seg(image)

        pil_image = Image.fromarray(color_seg)
        pil_image.save(destination_path)

        # Display the predicted segmentation
        display(pil_image)          

do_in_file_path(source_dir, get_and_save_segmentation)


df_images_as_arrays = pd.DataFrame(images_as_arrays)
df_images_as_arrays.to_pickle(f'{out_dir}/segmentation_results.pickle')